In [1]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


Kaggle credentials set.
Kaggle credentials successfully validated.


In [2]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

darshitsh_finetuned_unet_path = kagglehub.dataset_download('darshitsh/finetuned-unet')
darshitsh_finetuned_path = kagglehub.dataset_download('darshitsh/finetuned')
darshitsh_finetuned_v1_path = kagglehub.dataset_download('darshitsh/finetuned-v1')

print('Data source import complete.')


100%|██████████| 2.14M/2.14M [00:01<00:00, 1.75MB/s]

Extracting files...


100%|██████████| 2.14M/2.14M [00:01<00:00, 1.91MB/s]

Extracting files...


100%|██████████| 2.13M/2.13M [00:01<00:00, 1.96MB/s]

Extracting files...
Data source import complete.


In [3]:
!pip install fastapi uvicorn torch diffusers transformers accelerate pyngrok peft==0.5.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.6/85.6 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 123.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 99.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 62.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21

In [4]:
# Import libraries
from fastapi import FastAPI, HTTPException
from fastapi.responses import FileResponse
from pydantic import BaseModel
import torch
from diffusers import StableDiffusionPipeline
import uuid
import os
from google.colab import files
import nest_asyncio
from pyngrok import ngrok
import uvicorn
from safetensors.torch import load_file
import logging
from peft import PeftModel
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)
# Apply nest_asyncio to allow nested event loops (needed for Colab)
nest_asyncio.apply()


In [6]:
from fastapi import FastAPI, HTTPException
from fastapi.responses import FileResponse
from pydantic import BaseModel
import torch
import os
import uuid
import logging
from diffusers import StableDiffusionPipeline
from safetensors.torch import load_file
from peft import PeftModel

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

class ImageRequest(BaseModel):
    prompt: str
    negative_prompt: str = None
    num_inference_steps: int = 50
    guidance_scale: float = 7.5
    width: int = 512
    height: int = 512
    model: str = "finetuned"

# Create FastAPI app
app = FastAPI(title="Stable Diffusion API")

# Initialize stable diffusion model
model_id = "runwayml/stable-diffusion-v1-5"
pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16)

# # Path to the LoRA adapter model
unet_safetensor_path = darshitsh_finetuned_v1_path

if os.path.exists(unet_safetensor_path):
    try:
        def null_safety(images, **kwargs):
           return images, [False] * len(images)

        pipe.safety_checker = null_safety
        # Apply LoRA weights using PEFT
        pipe.unet = PeftModel.from_pretrained(pipe.unet, unet_safetensor_path)

        # Load the base model on GPU first
        pipe = pipe.to("cuda")
        print(f"Successfully loaded LoRA adapter from {unet_safetensor_path}")
    except Exception as e:
        print(f"Error loading LoRA adapter: {str(e)}")
        print("Will use default weights instead")
        # Make sure pipe is on GPU
        pipe = pipe.to("cuda")
else:
    logger.warning(f"File not found at {unet_safetensor_path}, using default weights")
    # Make sure pipe is on GPU
    pipe = pipe.to("cuda")

# Create a directory to store generated images
os.makedirs("generated_images", exist_ok=True)

@app.post("/generate-image/")
async def generate_image(request: ImageRequest):
    try:
        # Generate the image

        image = pipe(
            prompt=request.prompt,
            negative_prompt= request.negative_prompt,
            num_inference_steps = request.num_inference_steps,
            guidance_scale = request.guidance_scale,
            width= request.width,
            height = request.height
        ).images[0]


        # Save the image with a unique filename
        filename = f"generated_images/{uuid.uuid4()}.png"
        image.save(filename)

        # Return the image file
        return FileResponse(filename, media_type="image/png")

    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

@app.get("/")
async def root():
    return {"message": "Stable Diffusion API is running with LoRA adapter. Use /generate-image/ endpoint to create images."}

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

Successfully loaded LoRA adapter from /root/.cache/kagglehub/datasets/darshitsh/finetuned-v1/versions/1


In [7]:
!ngrok config add-authtoken 2w5ryKUhx0ecXhHvgn1sWHzinSG_K198KWdKxfDycTaL26Py

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
# Set up ngrok for public URL (since Colab doesn't expose ports directly)
!pip install pyngrok
ngrok_tunnel = ngrok.connect(8000)
print(f"Public URL: {ngrok_tunnel.public_url}")

# Run the FastAPI app
uvicorn.run(app, host="0.0.0.0", port=8000)

Public URL: https://9bb8-34-87-168-184.ngrok-free.app


INFO:     Started server process [391]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


INFO:     2601:19b:280:f500:1188:ab0e:f924:a0a1:0 - "GET / HTTP/1.1" 200 OK
INFO:     2601:19b:280:f500:1188:ab0e:f924:a0a1:0 - "GET / HTTP/1.1" 200 OK


  0%|          | 0/50 [00:00<?, ?it/s]

INFO:     2601:19b:280:f500:1188:ab0e:f924:a0a1:0 - "POST /generate-image/ HTTP/1.1" 200 OK
INFO:     2601:19b:280:f500:1188:ab0e:f924:a0a1:0 - "GET / HTTP/1.1" 200 OK
INFO:     2601:19b:280:f500:1188:ab0e:f924:a0a1:0 - "GET / HTTP/1.1" 200 OK
INFO:     2601:19b:280:f500:1188:ab0e:f924:a0a1:0 - "GET / HTTP/1.1" 200 OK
INFO:     2601:19b:280:f500:1188:ab0e:f924:a0a1:0 - "GET / HTTP/1.1" 200 OK


...

INFO:     2601:19b:280:f500:1188:ab0e:f924:a0a1:0 - "POST /generate-image/ HTTP/1.1" 200 OK
INFO:     2601:19b:280:f500:1188:ab0e:f924:a0a1:0 - "GET / HTTP/1.1" 200 OK
INFO:     2601:19b:280:f500:1188:ab0e:f924:a0a1:0 - "GET / HTTP/1.1" 200 OK
INFO:     2601:19b:280:f500:1188:ab0e:f924:a0a1:0 - "GET / HTTP/1.1" 200 OK
INFO:     2601:19b:280:f500:1188:ab0e:f924:a0a1:0 - "GET / HTTP/1.1" 200 OK
INFO:     2601:19b:280:f500:1188:ab0e:f924:a0a1:0 - "GET / HTTP/1.1" 200 OK
INFO:     2601:19b:280:f500:1188:ab0e:f924:a0a1:0 - "GET / HTTP/1.1" 200 OK
INFO:     2601:19b:280:f500:1188:ab0e:f924:a0a1:0 - "GET / HTTP/1.1" 200 OK
INFO:     2601:19b:280:f500:1188:ab0e:f924:a0a1:0 - "GET / HTTP/1.1" 200 OK
INFO:     2601:19b:280:f500:1188:ab0e:f924:a0a1:0 - "GET / HTTP/1.1" 200 OK
INFO:     2601:19b:280:f500:1188:ab0e:f924:a0a1:0 - "GET / HTTP/1.1" 200 OK
INFO:     2601:19b:280:f500:1188:ab0e:f924:a0a1:0 - "GET / HTTP/1.1" 200 OK
INFO:     2601:19b:280:f500:1188:ab0e:f924:a0a1:0 - "GET / HTTP/1.1" 200

  0%|          | 0/50 [00:00<?, ?it/s]

INFO:     2601:19b:280:f500:1188:ab0e:f924:a0a1:0 - "POST /generate-image/ HTTP/1.1" 200 OK
INFO:     2601:19b:280:f500:1188:ab0e:f924:a0a1:0 - "GET / HTTP/1.1" 200 OK
INFO:     2601:19b:280:f500:1188:ab0e:f924:a0a1:0 - "GET / HTTP/1.1" 200 OK
INFO:     2601:19b:280:f500:1188:ab0e:f924:a0a1:0 - "GET / HTTP/1.1" 200 OK
INFO:     2601:19b:280:f500:1188:ab0e:f924:a0a1:0 - "GET / HTTP/1.1" 200 OK


  0%|          | 0/50 [00:00<?, ?it/s]

INFO:     2601:19b:280:f500:1188:ab0e:f924:a0a1:0 - "POST /generate-image/ HTTP/1.1" 200 OK
INFO:     2601:19b:280:f500:1188:ab0e:f924:a0a1:0 - "GET / HTTP/1.1" 200 OK
